# Data Mining Project
### Giorgio Donati, g.donati24@studenti.unipi.it
### Pietro Francaviglia, p.francaviglia1@studenti.unipi.it
#### A.Y. 2021-2022


## Libraries and datasets

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from t3_constants import *
from t3_utility import *

In [24]:
with open(f'{PICKLE_FOLDER}{SELECTED}{USER_DF}.pkl', 'rb') as f:
    u_df = pickle.load(f)

with open(f'{PICKLE_FOLDER}{PURE}{SELECTED}{USER_DF}.pkl', 'rb') as f:
    pure_u_df = pickle.load(f)

In [29]:
u_df

,item_count,item_dist_count,max_items_per_b,price_entropy,max_item_dist_per_b,total_price,basket_count,category_count,shop_count,avg_items_per_basket,avg_baskets_per_d
user_id,,,,,,,,,,,
1,709,580,141,3.806662,123,630951.756741,45,48,30,15.755556,1.250000
2,903,669,158,4.168650,104,723773.163169,66,45,31,13.681818,1.404255
3,397,320,50,3.688879,43,488038.085519,40,41,27,9.925000,1.290323
4,932,680,214,3.891820,157,778783.061807,49,41,28,19.020408,1.400000
5,824,620,94,3.912023,82,699040.663917,50,44,23,16.480000,1.515152
...,...,...,...,...,...,...,...,...,...,...,...
819,610,500,68,3.931826,53,465303.064484,51,48,24,11.960784,1.342105
820,783,581,78,4.043051,66,664045.768047,57,49,28,13.736842,1.583333
821,646,473,77,3.871201,62,516218.782161,48,44,29,13.458333,1.371429


In [30]:
predictive_df = u_df[[ICNT, AIXB, PSUM, BCNT]]
predictive_df

,item_count,avg_items_per_basket,total_price,basket_count
user_id,,,,
1,709,15.755556,630951.756741,45
2,903,13.681818,723773.163169,66
3,397,9.925000,488038.085519,40
4,932,19.020408,778783.061807,49
5,824,16.480000,699040.663917,50
...,...,...,...,...
819,610,11.960784,465303.064484,51
820,783,13.736842,664045.768047,57
821,646,13.458333,516218.782161,48
